![alt text](./img/header.png)

# SEAWAT Exercise A: Henry Problem

The purpose of this exercise is to use SEAWAT to simulate the classic Henry problem. As described in class, some people consider there to be two cases of the Henry problem—each with different values for molecular diffusion. Code developers often use the Henry problem to test variable-density programs because an analytical solution exists for the problem (Simpson, 2004, is probably the best solution). In addition to Henry’s original paper, information for the Henry problem can be found in Langevin et al. (2003, page 34) and Guo and Langevin (2002, page 70).

## Part I. Setup Notebook

In [ ]:
# %load nbsetup.py
%matplotlib inline
from __future__ import print_function
import sys
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import flopy
import config

In [ ]:
# setup a model workspace directory
model_ws = os.path.join('work', 'exSEAWAT_A')
if not os.path.isdir(model_ws):
    os.mkdir(model_ws)

## Part II. Grid and Aquifer Properties

In [ ]:
# model grid information
nlay = 10
nrow = 1
ncol = 21
top = 1.
bot = 0.
dz = (top - bot) / nlay
botm = np.array([[z] * ncol for z in np.arange(1-dz, 1-(nlay + 1) * dz, -dz)]).reshape((nlay, nrow, ncol))
delr = np.array([0.1] * ncol)
delr[-1] = 0.01
delc = 1

In [ ]:
# temporal discretization
perlen = 0.15
nstp = 1
tsmult = 1.0
ttsmult = 1.1

In [ ]:
# Input variables
hk = 864.
vk = 864.
porosity = 0.35
ss = 0.0001
dmcoef = 1.629
Qinflow = 5.702
Csalt = 35.0001
Cfresh = 0.
densesalt = 1025.
densefresh = 1000.
denseslp = (densesalt - densefresh) / (Csalt - Cfresh)

## Part III. Boundary Conditions

In [ ]:
# Create the data needed to make the constant head package
chddata = []
for k in range(nlay):
    chddata.append([k, 0, ncol - 1, 1.0, 1.0])

In [ ]:
# Create the data needed to make the well package
weldata = []
wellQ = Qinflow / nlay
for k in range(nlay):
    weldata.append([k, 0, 0, wellQ])

In [ ]:
# create the ssm data
itype = flopy.mt3d.Mt3dSsm.itype_dict()
ssmdata = []
for rec in chddata:
    k = rec[0]
    i = rec[1]
    j = rec[2]
    ssmdata.append([k, i, j, Csalt, itype['CHD']])
for rec in weldata:
    k = rec[0]
    i = rec[1]
    j = rec[2]
    ssmdata.append([k, i, j, Cfresh, itype['WEL']])

In [ ]:
ssmdata = {0: ssmdata}
chddata = {0: chddata}
weldata = {0: weldata}

## Part IV. Creating the FloPy Objects 

In [ ]:
# create the flopy objects
modelname = 'henry'
m = flopy.seawat.Seawat(modelname, model_ws=model_ws, exe_name=config.swexe)

# modflow packages
dis = flopy.modflow.ModflowDis(m, nlay, nrow, ncol, nper=1, delr=delr,
                               delc=delc, laycbd=0, top=top,
                               botm=botm, perlen=perlen, nstp=nstp)
bas = flopy.modflow.ModflowBas(m)
lpf = flopy.modflow.ModflowLpf(m, hk=hk, vka=hk)
chd = flopy.modflow.ModflowChd(m, stress_period_data=chddata)
wel = flopy.modflow.ModflowWel(m, stress_period_data=weldata)
pcg = flopy.modflow.ModflowPcg(m, hclose=1.e-8)
oc = flopy.modflow.ModflowOc(m, stress_period_data={(0, 0): ['save head', 'save budget']},
                             compact=True)

# mt3d packages
btn = flopy.mt3d.Mt3dBtn(m, nprs=-1, prsity=porosity, sconc=Csalt, ifmtcn=0,
                         chkmas=False, nprobs=10, nprmas=10, dt0=1.e-4, ttsmult=ttsmult)
adv = flopy.mt3d.Mt3dAdv(m, mixelm=0)
dsp = flopy.mt3d.Mt3dDsp(m, al=0., trpt=1., trpv=1., dmcoef=dmcoef)
gcg = flopy.mt3d.Mt3dGcg(m, iter1=500, mxiter=1, isolve=1, cclose=1e-7)
ssm = flopy.mt3d.Mt3dSsm(m, stress_period_data=ssmdata)

# seawat packages
vdf = flopy.seawat.SeawatVdf(m, iwtable=0, densemin=0, densemax=0,
                             denseref=densefresh, denseslp=denseslp, firstdt=1e-3)

In [ ]:
# make a cross section plot of grid and boundaries
plt.figure(figsize=(10,5))
xs = flopy.plot.ModelCrossSection(model=m, line={'row': 0})
xs.plot_grid()
xs.plot_bc(package=wel, color='red')
xs.plot_bc(package=chd, color='k');

In [ ]:
# write the SEAWAT input files
m.write_input()

## Part V. Running the Model

In [ ]:
try:
    os.remove(os.path.join(model_ws, 'MT3D001.UCN'))
except:
    pass
m.run_model(silent=True)

## Part VI. Importing and Plotting Results

In [ ]:
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])
plt.figure(figsize=(10,5))
xs = flopy.plot.ModelCrossSection(model=m, line={'row': 0})
xs.plot_grid()
xs.plot_array(conc, vmin=Cfresh, vmax=Csalt)

## Part VI. Animate Results

In [ ]:
import time
from IPython.display import clear_output, display
animate = False
if animate:
    f = plt.figure(figsize=(10, 5))
    ax = plt.subplot(1, 1, 1)
    for i, t in enumerate(ucnobj.get_times()):
        conc = ucnobj.get_data(totim=times[i])
        ax.set_title("totim:{0:5.4f}".format(t))
        xs = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
        xs.plot_array(conc, vmin=Cfresh, vmax=Csalt)
        time.sleep(0.01)
        clear_output(True)
        display(f)
        ax.cla()
    print('Done.');

## Part VII. Questions

1. Can you run this problem without including the effects of fluid density?
2. Try setting the molecular diffusion value to zero and running the simulation.  Does this give us any insight into numerical dispersion?
2. Try running the modified Henry problem described by Simpson and Clement (2003, 2004), which requires cutting the inflow rate in half.
3. Experiment with different grid resolution.  How does this change the simulated salinity distribution?
4. What about temporal resolution?  Are the simulated salinity values affected by the size of the time step?
5. TVD is a powerful tranpsort solution scheme avaiable in MT3D.  It is specified in the advection package.  Can you figure out how to run this problem using TVD?
6. Time permitting, try looking at the simulated salinities using ModelViewer.

## Part VII. Cleanup

In [ ]:
# clean up
#if os.path.isdir(model_ws):
#    shutil.rmtree(model_ws)